<div style="display:flex;flex-direction:row;justify-content: space-evenly;">
<img src="tecnm.png" style="height:100px;"/>
<img src="itcolima.svg" style="width:100px"/>
</div>
<div style="display:flex;flex-direction:column;gap:20px;text-align:center">
<h1>Tecnológico Nacional de México campus Colima</h1>
<h2>Maestría en Sistemas Computacionales</h2>
<h2>Tecnologías de programación</h2>
<h2>Reto práctico 2 - Primitivas de sincronización</h2>
<h2>D. en C. Patricia Elizabeth Figueroa Millán</h2>
<h3>Angel Primitivo Vejar Cortés | G2146001 </h3>
<p style="text-align:right;">Villa de Álvarez, Colima - 08 de noviembre de 2022</p>
<p></p>
</div>

# Primitivas de sincronización

## Objetivo
Que el estudiante investigue y pueda realizar ejemplos aplicando las primitivas de sincronización para la sincronización de hilos y control de acceso a recursos compartidos.

## Metodología
Para el desarrollo de la actividad primero se relalizó una investigación documental por medio de los distintos posts de foros y documentación oficial de Python, para posteriormente realizar un ejemplo de cada primitiva de sincronización que son: 
- Lock
- Semaphore
- BoundedSemaphore
- Condition
- Event
- Barrier


## Materiales
Para el desarrollo de la actividad se utilizaron los siguientes materiales:
* Computadora con acceso a internet
* Editor de texto (Visual Studio Code)
* Aplicación jupyter notebook


## Desarrollo
A continuación se presenta una descripción breve de cada primitiva de sincronización así como un ejemplo de su uso.



### Primitiva de sincronización: Lock

La primitiva de sincronización Lock es una herramienta que permite controlar el acceso a una sección crítica de código, es decir, un bloque de código que debe ser ejecutado de manera exclusiva por un solo hilo a la vez. Para utilizar esta primitiva se debe crear un objeto de tipo Lock y utilizar los métodos acquire() y release() para controlar el acceso a la sección crítica [1].

En el siguiente ejemplo la sección crítica corresponde en acceder a una variable global e incrementar su valor en 1, para ello se utiliza un ciclo for con cada hilo que toma el lock para incrementarlo y lo suelta para pasar a la siguiente iteración.


In [1]:
import threading
from time import sleep
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)

# variable global
counter = 0

# método que incrementa el contador
def tarea(lock):
    global counter
    logging.debug('Iniciando')
    for _ in range(10):
        lock.acquire()
        counter += 1
        lock.release()
    logging.debug('Finalizando')

# creación de primitiva lock
lock = threading.Lock()

# crear hilos
for i in range(5):
    threading.Thread(target=tarea, args=(lock,),name="hilo "+str(i)).start()

sleep(0.1)

# esperar a que terminen los hilos
for i in threading.enumerate():
    if i.name.startswith("hilo"):
        i.join()

# imprimir el valor del contador
print(f'Valor del contador: {counter}')

2022-11-08 23:33:14,183 (hilo 0) Iniciando
2022-11-08 23:33:14,188 (hilo 1) Iniciando
2022-11-08 23:33:14,188 (hilo 0) Finalizando
2022-11-08 23:33:14,193 (hilo 2) Iniciando
2022-11-08 23:33:14,194 (hilo 1) Finalizando
2022-11-08 23:33:14,194 (hilo 3) Iniciando
2022-11-08 23:33:14,195 (hilo 4) Iniciando
2022-11-08 23:33:14,204 (hilo 2) Finalizando
2022-11-08 23:33:14,209 (hilo 3) Finalizando
2022-11-08 23:33:14,213 (hilo 4) Finalizando


Valor del contador: 50


Es posible obtener el mismo resultado utilizando la sentencia with, la cual se encarga de llamar a los métodos acquire() y release() de forma automática.

In [2]:
import threading
from time import sleep
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)

# variable global
counter = 0

# método que incrementa el contador
def tarea(lock):
    global counter
    logging.debug('Iniciando')
    for _ in range(10):
        with lock: # sustituye a lock.acquire() y lock.release()
            counter += 1
    logging.debug('Finalizando')

# creación de primitiva lock
lock = threading.Lock()

# crear hilos
for i in range(5):
    threading.Thread(target=tarea, args=(lock,),name="hilo "+str(i)).start()

sleep(0.1)

# esperar a que terminen los hilos
for i in threading.enumerate():
    if i.name.startswith("hilo"):
        i.join()

# imprimir el valor del contador
print(f'Valor del contador: {counter}')

2022-11-08 23:33:14,401 (hilo 0) Iniciando
2022-11-08 23:33:14,406 (hilo 1) Iniciando
2022-11-08 23:33:14,407 (hilo 0) Finalizando
2022-11-08 23:33:14,411 (hilo 2) Iniciando
2022-11-08 23:33:14,412 (hilo 3) Iniciando
2022-11-08 23:33:14,414 (hilo 4) Iniciando
2022-11-08 23:33:14,415 (hilo 1) Finalizando
2022-11-08 23:33:14,421 (hilo 2) Finalizando
2022-11-08 23:33:14,426 (hilo 3) Finalizando
2022-11-08 23:33:14,432 (hilo 4) Finalizando


Valor del contador: 50


## Primitiva de sincronización: Event

La primitiva de sincronización Event es una herramienta que permite sincronizar la ejecución de hilos, es decir, permite que un hilo espere a que otro hilo notifique que ha terminado de ejecutar una tarea. Para utilizar esta primitiva se debe crear un objeto de tipo Event y utilizar los métodos wait() y set() para controlar la sincronización de hilos [2].

In [3]:
import threading
from time import sleep
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)

# método que activa el evento
def make_ping(event):
    logging.debug('PING!!!')
    event.set()

# método que requiere un evento para continuar
def make_pong(event):
    # esperar por el evento
    logging.debug('//Esperando por PING')
    event.wait()
    logging.debug('PONG!!!')


# creación de evento
event = threading.Event()

# crear hilos

# se crea el hilo que requiere el evento
threading.Thread(target=make_pong, args=(event,),name="hilo pong").start()

sleep(1) # esperar un segundo para demostrar que el hilo pong espera por el evento

# se crea el hilo que activa el evento
threading.Thread(target=make_ping, args=(event,),name="hilo ping").start()


2022-11-08 23:33:14,638 (hilo pong) //Esperando por PING
2022-11-08 23:33:15,648 (hilo ping) PING!!!
2022-11-08 23:33:15,655 (hilo pong) PONG!!!


Es posible agregar un timeout al método wait() para que el hilo espere un tiempo determinado antes de continuar con su ejecución, con el objetivo de evitar que un hilo espere indefinidamente.

In [4]:
import threading
from time import sleep
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)
# método que activa el evento
def make_ping(event):
    logging.debug('PING!!!')
    event.set()

# método que requiere un evento para continuar
def make_pong(event,timeout):
    # esperar por el evento
    logging.debug('//Esperando por PING')
    res = event.wait(timeout)
    if res:
        logging.debug('PONG!!!')
    else:
        logging.debug('No se recibió PING a tiempo :(')


# creación de evento
event = threading.Event()

# crear hilos

# se crea el hilo que requiere el evento
threading.Thread(target=make_pong, args=(event,1),name="hilo pong").start()

sleep(3) # esperar un segundo para demostrar que el hilo pong espera por el evento

# se crea el hilo que activa el evento
threading.Thread(target=make_ping, args=(event,),name="hilo ping").start()


2022-11-08 23:33:15,767 (hilo pong) //Esperando por PING
2022-11-08 23:33:16,778 (hilo pong) No se recibió PING a tiempo :(
2022-11-08 23:33:18,777 (hilo ping) PING!!!


## Primitiva de sincronización: Condition

La primitiva de sincronización Condition es una herramienta que permite sincronizar la ejecución de hilos, es decir, permite que un hilo espere a que otro hilo notifique que ha terminado de ejecutar una tarea. Para utilizar esta primitiva se debe crear un objeto de tipo Condition y utilizar los métodos wait(), notify() y notify_all() para controlar la sincronización de hilos [3].

In [5]:
import threading
from time import sleep
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)

# método que requiere la condición
def comprar_pan(condicion):
    with condicion:
        logging.debug('Esperando por el pan')
        condicion.wait()
        print(threading.current_thread().name + ' compró el pan') 

# método que prepara el pan
def preparar_pan(condicion):
    logging.debug('Preparando pan')
    sleep(1)
    logging.debug('Pan listo')
    sleep(0.001)
    with condicion:
        condicion.notify_all()

# creación de condición
condicion = threading.Condition()

# crear hilos
threading.Thread(target=comprar_pan, args=(condicion,),name="hilo comprador 1").start()
threading.Thread(target=comprar_pan, args=(condicion,),name="hilo comprador 2").start()
threading.Thread(target=comprar_pan, args=(condicion,),name="hilo comprador 3").start()

sleep(1) # esperar un segundo para demostrar que los hilos compradores esperan por el pan

# se crea el hilo que prepara el pan
threading.Thread(target=preparar_pan, args=(condicion,),name="hilo panadero").start()

# join 
for i in threading.enumerate():
    if i.name.startswith("hilo"):
        i.join()

2022-11-08 23:33:18,879 (hilo comprador 1) Esperando por el pan
2022-11-08 23:33:18,888 (hilo comprador 2) Esperando por el pan
2022-11-08 23:33:18,894 (hilo comprador 3) Esperando por el pan
2022-11-08 23:33:19,902 (hilo panadero) Preparando pan
2022-11-08 23:33:20,916 (hilo panadero) Pan listo


hilo comprador 2 compró el pan
hilo comprador 3 compró el pan
hilo comprador 1 compró el pan


## Primitiva de sincronización: Semaphore

Un semáforo gestiona un contador interno que se reduce con cada llamada de acquire() y se incrementa con cada llamada de release(). El contador no puede bajar nunca por debajo de cero; cuando acquire() encuentra que es cero, se bloquea, esperando hasta que otro hilo llame a release()
Hay muchos casos en los que podemos querer permitir que más de un trabajador acceda a un recurso y al mismo tiempo limitar la cantidad total de accesos [4].

En el siguiente ejemplo se simula una conexión a una base de datos, en la cual se puede tener un máximo de 3 conexiones simultáneas, para ello se crea un semáforo con un valor inicial de 3 y se utiliza with para controlar el acceso a la sección crítica (en lugar de utilizar los métodos acquire() y release()), cada conexión y desconexión se imprime en pantalla para verificar que no se exceda el máximo de conexiones simultáneas.

In [6]:
import threading
from time import sleep
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)

# clase que administra la conexión de base de datos
class DBManager:
    def __init__(self) -> None:
        self.lock = threading.Lock()
        self.active_connections = []

    def connect(self, name):
        with self.lock:
            self.active_connections.append(name)
            print(f'Conexiones activas: {self.active_connections}')
            sleep(0.1)
    
    def disconnect(self, name):
        with self.lock:
            self.active_connections.remove(name)
            print(f'Conexiones activas: {self.active_connections}')
            sleep(0.1)

# método que simula la conexión a la base de datos
def handle_request(manager, semaphore):
    logging.debug('Esperando por conexión')
    with semaphore:
        name = threading.current_thread().name
        manager.connect(name)
        sleep(1)
        manager.disconnect(name)

# creación de semáforo
semaphore = threading.Semaphore(3)

# creación de administrador de base de datos
manager = DBManager()

# crear hilos
for i in range(10):
    threading.Thread(target=handle_request, args=(manager, semaphore),name="hilo "+str(i)).start()

# join 
for i in threading.enumerate():
    if i.name.startswith("hilo"):
        i.join()

2022-11-08 23:33:21,041 (hilo 0) Esperando por conexión
2022-11-08 23:33:21,048 (hilo 1) Esperando por conexión
2022-11-08 23:33:21,052 (hilo 2) Esperando por conexión
2022-11-08 23:33:21,054 (hilo 3) Esperando por conexión
2022-11-08 23:33:21,055 (hilo 4) Esperando por conexión
2022-11-08 23:33:21,059 (hilo 5) Esperando por conexión
2022-11-08 23:33:21,061 (hilo 6) Esperando por conexión


Conexiones activas: ['hilo 0']


2022-11-08 23:33:21,067 (hilo 7) Esperando por conexión
2022-11-08 23:33:21,068 (hilo 8) Esperando por conexión
2022-11-08 23:33:21,074 (hilo 9) Esperando por conexión


Conexiones activas: ['hilo 0', 'hilo 1']
Conexiones activas: ['hilo 0', 'hilo 1', 'hilo 2']
Conexiones activas: ['hilo 1', 'hilo 2']
Conexiones activas: ['hilo 2']
Conexiones activas: ['hilo 2', 'hilo 3']
Conexiones activas: ['hilo 3']
Conexiones activas: ['hilo 3', 'hilo 4']
Conexiones activas: ['hilo 3', 'hilo 4', 'hilo 5']
Conexiones activas: ['hilo 4', 'hilo 5']
Conexiones activas: ['hilo 4', 'hilo 5', 'hilo 6']
Conexiones activas: ['hilo 5', 'hilo 6']
Conexiones activas: ['hilo 5', 'hilo 6', 'hilo 7']
Conexiones activas: ['hilo 6', 'hilo 7']
Conexiones activas: ['hilo 6', 'hilo 7', 'hilo 8']
Conexiones activas: ['hilo 7', 'hilo 8']
Conexiones activas: ['hilo 7', 'hilo 8', 'hilo 9']
Conexiones activas: ['hilo 8', 'hilo 9']
Conexiones activas: ['hilo 9']
Conexiones activas: []


## Primitiva de sincronización: BoundedSemaphore

Un semáforo "encerrado" se puede visualizar como una versión más segura del semáforo puesto que este no permite que el contador interno baje por debajo de cero ni que suba por encima de su valor inicial (Si es 3 no se pueden subir a 4, cosa que sí es posible con el semáforo básico), en caso de utilizar release() cuando el contador es el valor inicial este hilo activa una excepción (error) [5].

Se parte del ejemplo anterior, en donde se limita la conexión a una base de datos, sin embargo, el último hilo intenta liberar el semáforo más de una vez, lo cual genera una excepción.

In [7]:
import threading
from time import sleep
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)
# clase que administra la conexión de base de datos
class DBManager:
    def __init__(self) -> None:
        self.lock = threading.Lock()
        self.active_connections = []

    def connect(self, name):
        with self.lock:
            self.active_connections.append(name)
            print(f'Conexiones activas: {self.active_connections}')
            sleep(0.1)
    
    def disconnect(self, name):
        with self.lock:
            self.active_connections.remove(name)
            print(f'Conexiones activas: {self.active_connections}')
            sleep(0.1)

# método que simula la conexión a la base de datos
def handle_request(manager, semaphore,breaker):
    logging.debug('Esperando por conexión')
    semaphore.acquire()
    name = threading.current_thread().name
    manager.connect(name)
    sleep(1)
    manager.disconnect(name)
    semaphore.release()

    if(breaker):
        semaphore.release()

# creación de semáforo
semaphore = threading.BoundedSemaphore(3)

# creación de administrador de base de datos
manager = DBManager()

# crear hilos
for i in range(10):
    threading.Thread(target=handle_request, args=(manager, semaphore, i == 9),name="hilo "+str(i)).start()

# join 
for i in threading.enumerate():
    if i.name.startswith("hilo"):
        i.join()

2022-11-08 23:33:26,206 (hilo 0) Esperando por conexión
2022-11-08 23:33:26,213 (hilo 1) Esperando por conexión
2022-11-08 23:33:26,217 (hilo 2) Esperando por conexión
2022-11-08 23:33:26,218 (hilo 3) Esperando por conexión
2022-11-08 23:33:26,221 (hilo 4) Esperando por conexión
2022-11-08 23:33:26,221 (hilo 5) Esperando por conexión
2022-11-08 23:33:26,223 (hilo 6) Esperando por conexión
2022-11-08 23:33:26,226 (hilo 7) Esperando por conexión
2022-11-08 23:33:26,226 (hilo 8) Esperando por conexión
2022-11-08 23:33:26,229 (hilo 9) Esperando por conexión


Conexiones activas: ['hilo 0']
Conexiones activas: ['hilo 0', 'hilo 1']
Conexiones activas: ['hilo 0', 'hilo 1', 'hilo 2']
Conexiones activas: ['hilo 1', 'hilo 2']
Conexiones activas: ['hilo 2']
Conexiones activas: ['hilo 2', 'hilo 3']
Conexiones activas: ['hilo 3']
Conexiones activas: ['hilo 3', 'hilo 4']
Conexiones activas: ['hilo 3', 'hilo 4', 'hilo 5']
Conexiones activas: ['hilo 4', 'hilo 5']
Conexiones activas: ['hilo 4', 'hilo 5', 'hilo 6']
Conexiones activas: ['hilo 5', 'hilo 6']
Conexiones activas: ['hilo 6']
Conexiones activas: ['hilo 6', 'hilo 7']
Conexiones activas: ['hilo 6', 'hilo 7', 'hilo 8']
Conexiones activas: ['hilo 7', 'hilo 8']
Conexiones activas: ['hilo 7', 'hilo 8', 'hilo 9']
Conexiones activas: ['hilo 8', 'hilo 9']
Conexiones activas: ['hilo 9']


Exception in thread hilo 9:
Traceback (most recent call last):
  File "c:\Users\ANGEL\AppData\Local\Programs\Python\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "c:\Users\ANGEL\AppData\Local\Programs\Python\Python39\lib\threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ANGEL\AppData\Local\Temp\ipykernel_28868\2687660180.py", line 38, in handle_request
  File "c:\Users\ANGEL\AppData\Local\Programs\Python\Python39\lib\threading.py", line 504, in release
    raise ValueError("Semaphore released too many times")
ValueError: Semaphore released too many times


Conexiones activas: []


## Primitiva de sincronización: Barrier

Una barrera es un objeto que permite sincronizar la ejecución de un grupo de hilos, es decir, permite que un grupo de hilos espere a que otro grupo de hilos notifique que ha terminado de ejecutar una tarea. Para utilizar esta primitiva se debe crear un objeto de tipo Barrier y utilizar los métodos wait() para controlar la sincronización de hilos [6].

In [8]:
import threading
from time import sleep
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s (%(threadName)-2s) %(message)s',
)
# método que simula la conexión a la base de datos
def tarea(barrier, time):
    sleep(time)
    logging.debug('Esperando a los panas')
    barrier.wait()
    logging.debug('El poder de la amistad!!!')

# creación de barrera
barrier = threading.Barrier(3)

# crear hilos
threading.Thread(target=tarea, args=(barrier, 2),name="hilo 1").start()
threading.Thread(target=tarea, args=(barrier, 4),name="hilo 2").start()
threading.Thread(target=tarea, args=(barrier, 5),name="hilo 3").start()

# join 
for i in threading.enumerate():
    if i.name.startswith("hilo"):
        i.join()

2022-11-08 23:33:33,351 (hilo 1) Esperando a los panas
2022-11-08 23:33:35,360 (hilo 2) Esperando a los panas
2022-11-08 23:33:36,355 (hilo 3) Esperando a los panas
2022-11-08 23:33:36,359 (hilo 3) El poder de la amistad!!!
2022-11-08 23:33:36,364 (hilo 1) El poder de la amistad!!!
2022-11-08 23:33:36,364 (hilo 2) El poder de la amistad!!!


# Resultados

Estos pueden ser observados en la consola de la aplicación jupyter notebook después de cada sección de código, en donde se imprime información que permite la compresión fácil de cada primitiva de sincronización, además, se recomienda visualizar el tiempo de diferencia entre las impresiones de mensajes puesto que algunas al estar separadas por un par de milisegundos representa una ejecución simultánea de hilos.

## Conclusiones

El uso de primitivas de sincronización permite programar de forma concurrente, aunque son útiles para proteger secciones críticas de código estas pueden tener un impacto en el rendimiento del programa (tiempod de ejecución), además, pueden ser utilizadas para sincronizar la ejecución de hilos, es decir, para que un hilo espere a que otro hilo notifique que ha terminado de ejecutar una tarea, tal es el caso de barrier que los hilos se esperan mutuamente para continuar con su ejecución o el caso de lock el cual evita más de una ejecución de una sección crítica de código.

## Bibliografía
1. https://www.bogotobogo.com/python/Multithread/python_multithreading_Synchronization_Lock_Objects_Acquire_Release.php
2. https://www.bogotobogo.com/python/Multithread/python_multithreading_Event_Objects_between_Threads.php
3. https://www.bogotobogo.com/python/Multithread/python_multithreading_Synchronization_Condition_Objects_Producer_Consumer.php
4. https://www.bogotobogo.com/python/Multithread/python_multithreading_Synchronization_Semaphore_Objects_Thread_Pool.php
5. https://stackoverflow.com/questions/48971121/what-is-the-difference-between-semaphore-and-boundedsemaphore
6. https://www.geeksforgeeks.org/barrier-objects-python/